In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 55.17403483390808  segundos


In [4]:
afi

,CEDULA_COD,ANIO,MES,CODSEC,SECTOR,DESTIPEMP,DESRELTRA,CODCII,PROVINCIA,CANTON,SALARIO,PORCENTAJE_AP
0,16260000,2003,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,D171,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,207.63,41.0
1,10788805,1994,4,R,PRIVADO,3-EMPRESA UNIPERSONAL / PEQUEÑA INDUSTRIA,06-CODIGO DEL TRABAJO - CT,M711011,IMBABURA,IBARRA,56.65,20.5
2,1866110,1991,12,R,PRIVADO,25-EMPLEADOR DOMESTICO,19-SERVICIO DOMESTICO - CT,P950,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,20.00,20.5
3,3579340,1986,5,R,PRIVADO,3-EMPRESA UNIPERSONAL / PEQUEÑA INDUSTRIA,06-CODIGO DEL TRABAJO - CT,A011301,EL ORO,MACHALA,56.65,20.5
4,10605673,2003,11,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K741409,IMBABURA,IBARRA,150.13,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...
68062265,13322434,2017,9,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,G45100102,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,400.00,20.6
68062266,10495276,2017,11,R,PRIVADO,3-EMPRESA UNIPERSONAL / PEQUEÑA INDUSTRIA,06-CODIGO DEL TRABAJO - CT,D289,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,900.00,20.6
68062267,1503916,2019,12,P,PUBLICO,"14-FUNCION EJECUTIVA, LEGISL, JUDICIAL Y ORGAN...",10-DOC. MAGISTERIO NOMBRAMIENTOS Y CONTRATOS,M802,CAÑAR,AZOGUES,817.00,20.6
68062268,16254330,2019,8,R,INDEPENDIENTES,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,109-GERENTE ADMINISTRADOR SIN RELACIÓN DE DEPE...,S96090705,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,394.00,17.6


In [5]:
afi.shape

(68062270, 12)

In [6]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi['CEDULA_COD'].value_counts()

In [11]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

112665

In [45]:
#Se filtran a las cedulas con menos de 60 imposicones 
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)]
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
afi_fil.shape

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2308\2270433710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afi_fil['FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')


(64756154, 13)

In [46]:
afi_fil

,CEDULA_COD,ANIO,MES,CODSEC,SECTOR,DESTIPEMP,DESRELTRA,CODCII,PROVINCIA,CANTON,SALARIO,PORCENTAJE_AP,FECHA
26481585,126,2004,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,147.45,20.5,2004-12-01
67442511,126,2005,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,41.0,2005-01-01
41179435,126,2005,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,2005-02-01
44033860,126,2005,3,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,2005-03-01
47099991,126,2005,4,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,2005-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896256,23594465,2017,11,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14045.72,20.6,2017-11-01
5107034,23594465,2017,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14164.25,20.6,2017-12-01
44333802,23594465,2018,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,12903.21,20.6,2018-01-01
38102314,23594465,2018,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,42432.21,20.6,2018-02-01


In [59]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA']].copy()

In [67]:
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index()
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
caract['MES_T'] = caract['CEDULA_COD'].map( afi['CEDULA_COD'].value_counts() )

In [69]:
caract

,CEDULA_COD,INI_HL,FIN_HL,MES_T
0,126,2004-12-01,2010-12-01,73
1,152,2004-07-01,2009-09-01,63
2,216,2004-07-01,2009-09-01,98
3,230,2004-12-01,2015-05-01,126
4,235,2004-08-01,2011-06-01,83
...,...,...,...,...
444112,23075971,2017-02-01,2022-10-01,66
444113,23402574,2010-01-01,2019-02-01,110
444114,23456125,2016-07-01,2022-03-01,69
444115,23503686,2004-09-01,2011-11-01,87


In [58]:
#Creación de fechas minimas y máximas de aportación
#Se considera como la fecha mínima la primera aportación y la máxima, la fecha ultima de aportación.
afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index()
    
    # Agrupar por 'CEDULA' y obtener la fecha mínima para cada grupo
    #min_fecha = df.groupby('CEDULA')['FECHA'].min().reset_index()

,CEDULA_COD,FECHA
0,126,2004-12-01
1,152,2004-07-01
2,216,2004-07-01
3,230,2004-12-01
4,235,2004-08-01
...,...,...
444112,23075971,2017-02-01
444113,23402574,2010-01-01
444114,23456125,2016-07-01
444115,23503686,2004-09-01


In [19]:

afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES']].copy()
#afi_car['ANIO_MIN'] = afi_car['CEDULA_COD'].map( afi_car.groupby('CEDULA_COD')['ANIO'].min() )
#afi_car['MES_MIN'] = afi_car['CEDULA_COD'].map( afi_car[ afi_car['ANIO']==grupo['ANIO_MIN']].groupby('CEDULA_COD')['MES'].min())

In [43]:
afi_sel.groupby('CEDULA_COD')['FECHA'].min()
#grupo['ANIO_MIN'] = grupo['CEDULA_COD'].map( grupo.groupby('CEDULA_COD')['ANIO'].min() )
#    grupo['MES_MIN']  = grupo['CEDULA_COD'].map(grupo[ grupo['ANIO']==grupo['ANIO_MIN']].groupby('CEDULA_COD')['MES'].min())

CEDULA_COD
126        2004-12-01
152        2004-07-01
216        2004-07-01
230        2004-12-01
235        2004-08-01
              ...    
23075971   2017-02-01
23402574   2010-01-01
23456125   2016-07-01
23503686   2004-09-01
23594465   2012-05-01
Name: FECHA, Length: 444117, dtype: datetime64[ns]